# DK_0911_post_allele_analysis_v02

Based on original code by Benjamin Schwessinger.

- Inputs: output from `DK_0911_defining_alleles_v02` & primary+haplotig (ph) protein/gene/cds .*fasta* files from `DK_0911_generate_fasta_files_from_gff3`.
- Programs: **MUSCLE**, **PAML**
- Purpose: generate and save a DataFrame containing dN/dS information (number of nonsynonymous substitutions per non-synonymous site to the number of synonymous substitutions per synonymous site), as well as Hamming & Levenshtein distances (measures of % identity). Also provides visualisations of some of this data.

#### Overview
1. Reads in the large allele DataFrames generated in `DK_0911_defining_alleles_v02` (i.e. proteinortho hits OR best blast hit) - see description header cell in that notebook for more information on which alleles are included in that DataFrame.
2. Filters the allele DataFrames based on %ID and %QCov (this can be set to filter only BLAST-identified alleles or both BLAST- and proteinortho-identified alleles) so that distance information is not calculated on an unnecessarily large number of alleles.
3. Calculates distance & dN/dS information, and saves this to an output file so that it does not have to be re-calculated (if for whatever reason, the inputs change so that dN/dS or distance information should change, this output file (`DK_0911_v0x_analysed_alleles.df`) should be deleted so that it can be re-generated.
4. Plots graphs of allele-type distribution (pie chart) and allele-type Levenshtein distances (measures of similarity) for different levels of allele-filtering (QCov/TCov/%ID/Levenshtein similarity).

NB:
- dN/dS information is currently not utilised in this script.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import shutil
from Bio import SeqIO
from Bio import AlignIO
import distance
import editdistance
import math
import subprocess
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import collections
import pybedtools
from sklearn.externals.joblib import Parallel, delayed
import itertools as it

In [3]:
#define variables here
GENOME_VERSION = 'v04'

BASE_PATH = '/home/benjamin/genome_assembly/Warrior/allele_analysis/%s/' % GENOME_VERSION

YN00_PATH = '/home/gamran/genome_analysis/Warrior/Richard/output/post_analysis/yn00.ctl'
BASE_OUT_PATH = os.path.join(BASE_PATH, 'post_analysis/')
ALLELE_PATH = os.path.join(BASE_PATH, 'allele_analysis/alleles_proteinortho_graph516/')
UNFILTERED_DF_PATH = os.path.join(BASE_PATH, \
    'allele_analysis/DK_0911_%s_p_ctg.DK_0911_%s_h_ctg.0.001.blastp.outfmt6.allele_analysis' \
                                  % (GENOME_VERSION, GENOME_VERSION))
GENOME_PATH = '/home/benjamin/genome_assembly/Warrior/genome_%s/' % GENOME_VERSION
FIGURE_PATH = os.path.join(BASE_OUT_PATH, 'figures')

GENOME = 'DK_0911_%s' % GENOME_VERSION
P_GENOME = GENOME + '_p_ctg'
H_GENOME = GENOME + '_h_ctg'

threads = 8

# Base filtering so that distance calculations are not performed on all allele pairs.
# Distance calculations will only be performed on allele pairs above the defined cutoffs.
# Note that proteinortho alleles will not be affected (this can be changed in the filterAlleleDf function).
BASE_QCOV_CUTOFF = 0
BASE_TCOV_CUTOFF = 0
BASE_PCTID_CUTOFF = 0

P_PROTEINS_FASTA = os.path.join(GENOME_PATH, P_GENOME + '.protein.fa')

PAML_PATH = os.path.join(BASE_OUT_PATH, 'paml')
if not os.path.exists(BASE_OUT_PATH):
    os.mkdir(BASE_OUT_PATH)
if not os.path.exists(FIGURE_PATH):
    os.mkdir(FIGURE_PATH)
if not os.path.exists(PAML_PATH):
    os.mkdir(PAML_PATH)
    shutil.copy2(YN00_PATH, PAML_PATH)

In [4]:
#for homozygous region analysis
COV_PATH = '/home/benjamin/genome_assembly/Warrior/COV'
homo_bed_fh = os.path.join(COV_PATH, 'DK0911_v04_ph_ctg.ph_p_homo_cov.bed')
anno_gff_p_fh = os.path.join(GENOME_PATH, 'DK_0911_v04_p_ctg.anno.gff3')

In [5]:
PH_PROTEIN_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.protein.fa')
PH_GENE_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.gene.fa')
PH_CDS_FASTA = os.path.join(GENOME_PATH, GENOME + '_ph_ctg.cds.fa')

In [6]:
#for finalizing the protein ortho single analysis
proortho_fh = os.path.join(BASE_PATH, 'proteinortho', 'ph_ctg_516.proteinortho')
proorthograph_fh = os.path.join(BASE_PATH, 'proteinortho', 'ph_ctg_516.proteinortho-graph')
poff_graph_fh = os.path.join(BASE_PATH, 'proteinortho', 'ph_ctg_516.poff-graph')
assert(os.path.exists(proortho_fh))
assert(os.path.exists(proorthograph_fh))
assert(os.path.exists(poff_graph_fh))

In [7]:
def getInterHaplotypeParaloges(proorthograph_fh,poff_graph_fh):
    """This function compares the proteinorth (po) and po synteny output. 
    It returns all Target+Query pairs that are only found in po as list. 
    It also checks if there are pairs only found in poff."""
    graph_header = ['Target', 'Query', 'evalue_ab', 'bitscore_ab', 'evalue_ba', 'bitscore_ba', 'same_strand' , 'simscore']
    poff_df = pd.read_csv(poff_graph_fh, sep='\t', header=None, names=graph_header, comment='#' )
    po_df = pd.read_csv(proorthograph_fh, sep='\t', header=None, names=graph_header, comment='#' )
    poff_paring = (poff_df.Target+ '+' + poff_df.Query)
    po_paring = (po_df.Target+ '+' + po_df.Query)
    interhaplotypeparaloges_l = list(set(po_paring) - set(poff_paring))
    poff_only_pairs = list(set(poff_paring) - set(po_paring))
    if len(poff_only_pairs) > 0:
        print("Check poff and proteinortho output as conceptually all poff pairs should be in included in the proteinortho output.\nThese are the pairs:")
        print(poff_only_pairs)
    return interhaplotypeparaloges_l

In [8]:
def assign_INP(aTargetplusaQuery, proorthograph_fh = proorthograph_fh,poff_graph_fh = poff_graph_fh ):
    """
    Returns a series of of True/False values if a aTarget+aQuery pair are element identified as 
    Inter haplotype paraloge.
    Input:
        proorthograh_fh, proteinortho graph output file handle that has been run including the single flag
        queries, series of protein queries used for the blast search.
        poff_graph_fh, proteinortho graph output file handle that has been run including the synteny and
        single flag
        aTargetplusaQuery, series of protein aTargets + aQuiers used for the blast search.
    Outupt:
        Returns a True/False series with the aTargets + aQuiers as index.
    
    """

    interhaplotypeparaloges_l = getInterHaplotypeParaloges(proorthograph_fh,poff_graph_fh)
    interhaplotypeparaloges_s = pd.Series([False]*len(aTargetplusaQuery), index=aTargetplusaQuery)
    interhaplotypeparaloges_s.loc[interhaplotypeparaloges_s.index.isin(interhaplotypeparaloges_l)] = True
    #print(len(Unphasedgenes))
    return interhaplotypeparaloges_s

In [9]:
def getProOrthsingles(proortho_fh):
    """Parses out all proteinortho singles from a proteinortho output file.
    Returns a list of these singles."""
    with open(proortho_fh) as file:
        id_pattern = r'(evm\S*)'
        regex = re.compile(id_pattern)
        po_single_list = []
        for line in file:
            if '*' in line:
                po_single_list.append(regex.search(line).groups()[0])
            else:
                continue
    return po_single_list

In [10]:
def assign_PO_single(queries, proortho_fh=proortho_fh):
    """
    Returns a series of of True/False values if a query element identified as single.
    Input:
        proortho_fh, proteinortho output file handle that has been run including the single flag
        queries, series of protein queries used for the blast search.
    Outupt:
        Returns a True/False series with the queries as index.
    
    """

    PO_singles = getProOrthsingles(proortho_fh)
    PO_singles_s = pd.Series([False]*len(queries), index=queries)
    PO_singles_s.loc[PO_singles_s.index.isin(PO_singles)] = True
    #print(len(Unphasedgenes))
    return PO_singles_s

In [11]:
def allgffTogenegff(gff_fh, write_out=True):
    '''Converts at complete gff to a gene gff only and writtes it out.'''
    gene_gff = pd.read_csv(gff_fh, sep='\t', header=None)
    gene_gff = gene_gff[gene_gff[2] == 'gene']
    gene_gff.reset_index(drop=True, inplace=True)
    gene_gff.to_csv(gff_fh.replace('anno', 'gene'), sep='\t', header=False, index=False)
    return gene_gff

In [12]:
def col_8_id(x):
    '''Function that pulls out the ID from the 9th column of a df.'''
    pattern = r'ID=([a-zA-Z0-9_.]*);'
    regex = re.compile(pattern)  
    m = regex.search(x)
    match = m.groups()[0].replace('TU', 'model')
    if match.startswith('cds.'):
        match = match[4:]
    if 'exon' in match:
        _list = match.split('.')
        match = '.'.join(_list[:-1])
    return match

In [206]:
def assignMatchType(allele_source, overlap, no_overlap):
    if allele_source == np.nan:
        return allele_source
    
    s = allele_source + '_'
    
    if overlap ==  True:
        s += 'overlap'
    elif no_overlap == True:
        s += 'no_overlap'
    else: # different_pcontig
        s += 'unlinked'
    return s

def reduceGroups(g):
    '''returns the best hit based on e-value and BitScore per group'''
    if len(g) == 1:
        return g
    tmp_g = g[g['e-value'] == g['e-value'].min()]
    if len(tmp_g) == 1:
        return tmp_g
    return tmp_g[tmp_g['BitScore'] == tmp_g['BitScore'].max()]

def filterAlleleDf(alleleDf, qCov, tCov, pctId, levSim, leavePO=False):
    if leavePO:
        no_PO_df = alleleDf[(alleleDf['allele_source'] == 'h_rBLAST') | (alleleDf['allele_source'] == 'BLAST')]
        PO_df = alleleDf[alleleDf['allele_source'] == 'PO']

        filtered_no_PO_df = filterAlleleDf(no_PO_df, qCov, tCov, pctId, levSim)
        return filtered_no_PO_df.append(PO_df, ignore_index=True)
    
    if qCov:
        alleleDf = alleleDf[alleleDf['QCov'] > qCov]
    if tCov:
        alleleDf = alleleDf[alleleDf['TCov'] > tCov]
    if pctId:
        alleleDf = alleleDf[alleleDf['PctID'] > pctId]
    if levSim:
        levDist = (100-levSim)/100.0
        alleleDf = alleleDf[alleleDf['protein_levenshtein'] < levDist]

    return alleleDf

In [14]:
def geneUnphased(Gene_gff_fh, Homo_cov_bed_fh ):
    """
    Returns a list of all genes that are unphased.
    
    Input: * Fh for annotation gff file
           * Fh for Homo_cov_bed_fh
    Output: A set of gene IDs that are unphases
    """
    geneGff_bed = pybedtools.BedTool(Gene_gff_fh)
    homo_p_bed = pybedtools.BedTool(Homo_cov_bed_fh)
    gene_ids_ph_p_homo = []
    for x in geneGff_bed.intersect(homo_p_bed, f=0.4):
        y = col_8_id(x[8])
        gene_ids_ph_p_homo.append(y)
    gene_ids_ph_p_homo = set(gene_ids_ph_p_homo)
    return gene_ids_ph_p_homo
    

In [15]:
def assign_unphased(queries, gff_fh=anno_gff_p_fh, homo_bed_fh=homo_bed_fh):
    """
    Takes a series of gene models and checks if they overlap with the
    homo_bed dataframe. The minimum overlap is f=0.4 with the intersect function.
    
    TO DO: For now the homo_bed only includes primary gene models. Should be changed.
    Input:  
        queries, pd.Series of gene names. 
        gff_fh, file handle of gff file.
        homo_bed_dfh, file handle of bed file that represents non-phased regions.
    
    """
    if '.gene' not in gff_fh: 
        _ = allgffTogenegff(gff_fh)
    Unphasedgenes = geneUnphased(gff_fh.replace('anno', 'gene'), homo_bed_fh)
    unphased_s = pd.Series([False]*len(queries), index=queries)
    unphased_s.loc[unphased_s.index.isin(Unphasedgenes)] = True
    #print(len(Unphasedgenes))
    return unphased_s

In [16]:
def getFastaDict(fastaFile):
    d = {}
    for gene in SeqIO.parse(fastaFile, 'fasta'):
        d[gene.id] = gene
    return d

In [17]:
def writeAllelicFasta(alleleOne, alleleTwo, alleleType, outPath):
    '''writes fasta file containing fasta information for two alleles
    in the outPath'''
    assert(alleleType.upper() in ['CDS', 'GENE', 'PROTEIN'])
    
    seqRecordDict = globals()['SEQRECORD_' + alleleType.upper() + '_DICT']
    try:
        alleleSeqRecords = [seqRecordDict[alleleOne], seqRecordDict[alleleTwo]]
    except KeyError:
        print(alleleOne)
        print(alleleTwo)
        print(alleleType)
        sys.exit()
    with open(os.path.join(outPath, alleleType.lower() + '.fa'), 'w') as outFile:
        SeqIO.write(alleleSeqRecords, outFile, 'fasta')
    return True

def writeAlignmentScript(alleleOutPath, scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'a') as outFile:
        print('cd %s' % alleleOutPath, file=outFile)
        print('/home/gamran/anaconda3/muscle3.8.31_i86linux64 -clwstrict -in protein.fa -out protein.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl -output paml protein.aln cds.fa > cds_codon.aln', file=outFile)
        print('perl /home/gamran/anaconda3/pal2nal.v14/pal2nal.pl protein.aln cds.fa > cds_codon.clustal', file=outFile)
        print('cp %s/yn00.ctl ./' % PAML_PATH, file=outFile)
        print('/home/gamran/anaconda3/paml4.9g/bin/yn00', file=outFile)
    return True

In [18]:
def prepareAlignmentBashScript(scriptLoc = os.path.join(PAML_PATH, 'paml_script.sh')):
    with open(scriptLoc, 'w') as pamlScript:
        print('#!/bin/bash', file=pamlScript)

    for index, [Query, Target] in alleleDf.iloc[:, :2].iterrows():
        #if we don't have a blast hit skip.
        if pd.isnull(Target):
            continue
        else:
            alleleOutPath = os.path.join(PAML_PATH, '%s_%s' % (Query, Target))
            if not os.path.exists(alleleOutPath):
                os.mkdir(os.path.join(PAML_PATH, '%s_%s' % (Query, Target)))

            writeAllelicFasta(Query, Target, 'CDS', alleleOutPath)
            writeAllelicFasta(Query, Target, 'PROTEIN', alleleOutPath)

            writeAlignmentScript(alleleOutPath, os.path.join(PAML_PATH, 'paml_script.sh'))

In [19]:
def assignDistancesToAlleles(folder, alignmentFile, alleleType):
    '''Adds Hamming and Levenshtein distance columns to an allele pair
    (indexed by 'folder' name) in df'''
    #print(folder)
    if pd.isnull(folder):
        return np.nan, np.nan
    assert(alleleType.upper() in ['PROTEIN', 'CDS', 'GENE'])
    seq1, seq2 = AlignIO.read(open(alignmentFile, 'r'), format='clustal', seq_count=2)
    seq1 = str(seq1.seq).upper()
    seq2 = str(seq2.seq).upper()
    assert(len(seq1) == len(seq2))
    return editdistance.eval(seq1, seq2)/len(seq1), distance.hamming(seq1, seq2, normalized=True)

def assignDistancesToAllAlleles(df_folder_index, all_folders, tmp_path, suffix):
    """
    Reads in the index that contains the folder pairings for the alignements.
    Returns a protein_df and CDS_df that contain the hamming and levenshtein distance each.
    """
    cleaned_index = [x for x in df_folder_index if x in all_folders]
    count = 0
    total = len(df_folder_index)
    percentDone = 0
    protein_lev_dict = {}
    protein_ham_dict = {}
    CDS_lev_dict = {}
    CDS_ham_dict = {}
    
    #print("Calculating distances and adding them to the allele DataFrame...")
    
    for folder in cleaned_index:
        if pd.isnull(folder):
            proteinAlignmentFile = ''
            cdsAlignmentFile = ''
        else:
            proteinAlignmentFile = os.path.join(PAML_PATH, folder, 'protein.aln')
            cdsAlignmentFile = os.path.join(PAML_PATH, folder, 'cds_codon.clustal')
        #here the nan get overwritten. This doesn't matter though as they are all
        #nan anyway.
        protein_lev_dict[folder], protein_ham_dict[folder]  = \
        assignDistancesToAlleles(folder, proteinAlignmentFile, 'PROTEIN')
        CDS_lev_dict[folder], CDS_ham_dict[folder]  = \
        assignDistancesToAlleles(folder, cdsAlignmentFile, 'CDS')

        count += 1
        #if round(count/total * 100) > percentDone:
            #percentDone = round(count/total * 100)
            #print("%s%% complete" % percentDone)
            
    newdf_columns=['protein_hamming', 'protein_levenshtein', 'cds_hamming',
       'cds_levenshtein']
    if len(protein_ham_dict) > 0:
        df = pd.DataFrame([protein_ham_dict,protein_lev_dict,CDS_ham_dict,CDS_lev_dict]).T
        df.rename(columns=dict(zip(df.columns,newdf_columns)),inplace=True)
        out_name = os.path.join(tmp_path, '%s_%s.%s' % (df.index[0],df.index[-1],suffix))
        df.round(4).to_csv(out_name, sep='\t')

In [20]:
def parse_dNdS_to_df(line, folder):
    """
    Function that parses out dN and dS of a yn00 file and calls the 
    assign_dNdS function. Therefore returns a single element
    pd.Series with folder name as index.
    """
    dN = re.findall(r'dN = [-| ]?(.*) w', line)[0]
    dS = re.findall(r'dS = [-| ]?(.*) dN', line)[0]
    return assign_dNdS(dN, dS, folder)

def assign_dNdS(dN, dS, folder):
    '''
    Function that cacluates the dN/dS ratio an returns it as a series usind the folder as index.
    Input: dN, dS, folder(name)
    Output: single element pd.Series with folder name as index.
    '''
    if float(dS) > 0:
        series = pd.Series([float(dN)/float(dS)], index=[folder])
    else:
        series = pd.Series([np.nan], index=[folder])
    return series

def assign_dNdS_to_all_alleles(folder_index, all_folders, tmp_path, suffix):
    """Function that parses out the different dN/dS ratios from a yn.out file for a list/index
    of folders that contain these yn.out files. The output dataframe is saved to tmp folder using
    the suffix.
    Input:  folder_index, list or index where to find the yn.out for each pairing.
            all_folders, are all possible folders. This is used to filter out nan and so from
                paralizing etc.
            tmp_path, is the path were 
    Output: Saved out tmp df with the suffix as file ending.
    
    """
    cleaned_index = [x for x in folder_index if x in all_folders]
    #print(cleaned_index)
    yn00_s = pd.Series([], name='yn00_dN/dS')
    LWL85_s = pd.Series([], name='LWL85_dN/dS')
    LWL85m_s = pd.Series([], name='LWL85m_dN/dS')
    LPB93_s = pd.Series([], name='LPB93_dN/dS')
    #header = ['folder','yn00_dN/dS', 'LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS']
    #append these list
    for folder in cleaned_index:
        alleleYn = os.path.join(PAML_PATH, folder,'yn.out')
        with open(alleleYn, 'r') as ynOut:
            #now loop over the lines and parse out stuff
            for i, line in enumerate(ynOut):
                if line.startswith('seq. seq. ') and i > 0:
                    next(ynOut) # we want the line that is two after the line starting with 'seq. seq '
                    dataLine = next(ynOut)
                    dN = dataLine.split('+-')[0].rstrip().split(' ')[-1]
                    dS = dataLine.split('+-')[1].rstrip().split(' ')[-1]
                    yn00_s = yn00_s.append(assign_dNdS(dN, dS, folder))
                elif line.startswith('LWL85:') and 'nan' not in line:
                    LWL85_s = LWL85_s.append(parse_dNdS_to_df(line, folder))
                elif line.startswith('LWL85m:') and 'nan' not in line:
                    LWL85m_s= LWL85m_s.append(parse_dNdS_to_df(line, folder))
                elif line.startswith('LPB93:') and 'nan' not in line:
                    LPB93_s =LPB93_s.append(parse_dNdS_to_df(line, folder))
                else:
                    continue
    out_df = pd.concat([yn00_s.round(4),LWL85_s.round(4),\
                        LWL85m_s.round(4), LPB93_s.round(4)],axis =1)
    new_columns = ['yn00_dN/dS','LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS' ]
    if len(out_df) > 0:
        out_df.rename(columns=dict(zip(out_df.columns, new_columns)), inplace=True)
        out_name = os.path.join(tmp_path, '%s_%s.%s' % (out_df.index[0],out_df.index[-1],suffix))
        out_df.to_csv(out_name, sep='\t')
    #return out_df

In [21]:
def checkPamlFilesExist(alleleDf):
    '''loops through all folder names in alleleDf.index to check if their PAML files have
    all been generated in those folders. refDict is based on the contents of a folder
    that was known to be run successfully.'''
    refDict = {'aln': 2,
     'clustal': 1,
     'ctl': 1,
     'dN': 1,
     'dS': 1,
     'fa': 2,
     'out': 1,
     'rst': 1,
     'rst1': 1,
     'rub': 1,
     't': 1}
    for file in (x for x in alleleDf.index if not pd.isnull(x)):
        if not os.path.exists(os.path.join(PAML_PATH, file)):
            return False
        discrepancies = getDiscrepancies(os.path.join(PAML_PATH, file), refDict)
        if discrepancies != '':
            print(discrepancies)
            return False
    return True

In [22]:
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return it.zip_longest(*args, fillvalue=fillvalue)

In [71]:
def combineTmpToDf(header, suffix, tmp_path, clean=True):
    """Combines the files of a temporary folder into a dataframe based on the tmp files
    suffix. Returns combined dataframe. And cleans up if needed."""
    tmp_assigneddfs_fh = [os.path.join(tmp_path, file) for file in os.listdir(tmp_path)\
                         if file.endswith(suffix) ]
    print(len(tmp_assigneddfs_fh))
    tmp_df = pd.DataFrame(columns=header)
    for df_fh in tmp_assigneddfs_fh:
        tmp_df = pd.concat([tmp_df, pd.read_csv(df_fh, index_col = 0, sep='\t')])
    if clean == True:
        #now clean up again
        for file in tmp_assigneddfs_fh:
            os.remove(file)
    return tmp_df


In [207]:
hFullAlleleDf = pd.read_csv(os.path.join(ALLELE_PATH, '%s.full_df.alleles' % H_GENOME), header=0, sep='\t')
hFullAlleleDf['matchType'] = pd.Series([np.nan]*len(hFullAlleleDf.index), index=hFullAlleleDf.index)
hFullAlleleDf['matchType'] = hFullAlleleDf.apply(lambda row: assignMatchType(row['allele_source'], row['t_contig == h_contig_overlap'], row['q_contig == t_contig']), axis=1)
pFullAlleleDf = pd.read_csv(os.path.join(ALLELE_PATH, '%s.full_df.alleles' % P_GENOME), header=0, sep='\t')
pFullAlleleDf['matchType'] = pd.Series([np.nan]*len(pFullAlleleDf.index), index=pFullAlleleDf.index)
pFullAlleleDf['matchType'] = pFullAlleleDf.apply(lambda row: assignMatchType(row['allele_source'], row['t_contig == h_contig_overlap'], row['q_contig == t_contig']), axis=1)

TypeError: ("unsupported operand type(s) for +: 'float' and 'str'", 'occurred at index 10793')

In [211]:
pFullAlleleDf.head()

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,...,TLgth,TCov,q_contig,t_contig,q_contig == t_contig,p_protein,h_contig_overlap,t_contig == h_contig_overlap,allele_source,matchType
0,evm.model.pcontig_000.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,pcontig_000,False,False,NaN,False,False,BLAST,BLAST
1,evm.model.pcontig_000.100,evm.model.hcontig_072_002.3,25.00,240.0,155.0,8.0,110.0,330.0,125.0,358.0,...,768.0,31.250000,pcontig_000,hcontig_072_002,False,evm.model.pcontig_000.100,['hcontig_000_058'],False,BLAST,BLAST
2,evm.model.pcontig_000.1001,evm.model.hcontig_029_007.1,70.27,37.0,11.0,0.0,130.0,166.0,52.0,88.0,...,90.0,41.111111,pcontig_000,hcontig_029_007,False,evm.model.pcontig_000.1001,['hcontig_000_078'],False,BLAST,BLAST
3,evm.model.pcontig_000.1002,evm.model.hcontig_003_006.10,96.88,128.0,4.0,0.0,1.0,128.0,83.0,210.0,...,210.0,60.952381,pcontig_000,hcontig_003_006,False,evm.model.pcontig_000.1002,['hcontig_000_078'],False,BLAST,BLAST
4,evm.model.pcontig_000.1003,evm.model.hcontig_003_006.9,96.95,164.0,5.0,0.0,1.0,164.0,345.0,508.0,...,508.0,32.283465,pcontig_000,hcontig_003_006,False,evm.model.pcontig_000.1003,['hcontig_000_078'],False,BLAST,BLAST


In [63]:
# filter out haplotig proteins that already have alleles identified by BLAST or proteinortho.
hFullAlleleDf = hFullAlleleDf[(~hFullAlleleDf['Query'].isin(pFullAlleleDf['Target']))]
pFullAlleleDf['aQuery'] = pFullAlleleDf['Query']
pFullAlleleDf['aTarget'] = pFullAlleleDf['Target']
hFullAlleleDf['aQuery'] = hFullAlleleDf['Target']
hFullAlleleDf['aTarget'] = hFullAlleleDf['Query']

In [64]:
phFullAlleleDf = pFullAlleleDf.append(hFullAlleleDf)

In [65]:
SEQRECORD_PROTEIN_DICT = getFastaDict(PH_PROTEIN_FASTA)
SEQRECORD_GENE_DICT = getFastaDict(PH_GENE_FASTA)
SEQRECORD_CDS_DICT = getFastaDict(PH_CDS_FASTA)

In [104]:
alleleDf = phFullAlleleDf.copy()
alleleDf['folder'] = alleleDf.Query + '_' + alleleDf.Target
alleleDf.set_index('folder', inplace=True)
# assert(len(alleleDf) == len(overlapDf) + len(noOverlapDf) + len(diffContigDf) + len(manualAssignDf))

In [84]:
alleleDf.head()

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,...,q_contig,t_contig,q_contig == t_contig,p_protein,h_contig_overlap,t_contig == h_contig_overlap,allele_source,matchType,aQuery,aTarget
folder,,,,,,,,,,,,,,,,,,,,,
NaN,evm.model.pcontig_000.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,pcontig_000,False,False,NaN,False,False,BLAST,BLAST,evm.model.pcontig_000.10,NaN
evm.model.pcontig_000.100_evm.model.hcontig_072_002.3,evm.model.pcontig_000.100,evm.model.hcontig_072_002.3,25.00,240.0,155.0,8.0,110.0,330.0,125.0,358.0,...,pcontig_000,hcontig_072_002,False,evm.model.pcontig_000.100,['hcontig_000_058'],False,BLAST,BLAST,evm.model.pcontig_000.100,evm.model.hcontig_072_002.3
evm.model.pcontig_000.1001_evm.model.hcontig_029_007.1,evm.model.pcontig_000.1001,evm.model.hcontig_029_007.1,70.27,37.0,11.0,0.0,130.0,166.0,52.0,88.0,...,pcontig_000,hcontig_029_007,False,evm.model.pcontig_000.1001,['hcontig_000_078'],False,BLAST,BLAST,evm.model.pcontig_000.1001,evm.model.hcontig_029_007.1
evm.model.pcontig_000.1002_evm.model.hcontig_003_006.10,evm.model.pcontig_000.1002,evm.model.hcontig_003_006.10,96.88,128.0,4.0,0.0,1.0,128.0,83.0,210.0,...,pcontig_000,hcontig_003_006,False,evm.model.pcontig_000.1002,['hcontig_000_078'],False,BLAST,BLAST,evm.model.pcontig_000.1002,evm.model.hcontig_003_006.10
evm.model.pcontig_000.1003_evm.model.hcontig_003_006.9,evm.model.pcontig_000.1003,evm.model.hcontig_003_006.9,96.95,164.0,5.0,0.0,1.0,164.0,345.0,508.0,...,pcontig_000,hcontig_003_006,False,evm.model.pcontig_000.1003,['hcontig_000_078'],False,BLAST,BLAST,evm.model.pcontig_000.1003,evm.model.hcontig_003_006.9


In [89]:
os.chdir('/home/gamran/genome_analysis/Warrior/Richard/scripts')
%run file_counting.ipynb

In [144]:
def main(alleleDf = alleleDf):
    prepareAlignmentBashScript(os.path.join(PAML_PATH, 'paml_script.sh'))
    
    # if already run before, comment out this line
    print("Checking whether all PAML files already exist in %s..." % PAML_PATH)
    if checkPamlFilesExist(alleleDf):
        print('PAML appears to have been run to completion previously. Therefore, it will not be run this time.')
    else:
        'Not all files generated by PAML appear to exist. Running PAML now (this may take some time)...'
        !bash {os.path.join(PAML_PATH, 'paml_script.sh')}
        print('PAML finished running.')

    analysedAllelesPath = os.path.join(BASE_OUT_PATH, GENOME+'_analysed_alleles.df')    
    alleleDf.to_csv(analysedAllelesPath, sep='\t')
    #dataframe where the index is not 'NaN'
    noNANdf = alleleDf.loc[~alleleDf.index.isnull(),:].copy()
    all_folders = noNANdf.index
    #generate a tmp folder for the parallized analysis
    tmp_path = os.path.join(BASE_OUT_PATH, 'tmp')
    if not os.path.exists(tmp_path):
        os.mkdir(tmp_path)
    #assign the distances
    dist_suffix = 'distdf_tmp'
    #do parallized analysi
    Parallel(n_jobs=threads)(delayed(assignDistancesToAllAlleles)(list(folder_index_list),all_folders,tmp_path, dist_suffix)\
                       for folder_index_list in grouper(noNANdf.index, 100, np.nan))
    distdf_header = ['protein_hamming', 'protein_levenshtein', 'cds_hamming',
       'cds_levenshtein']
    distdf = combineTmpToDf(distdf_header, dist_suffix, tmp_path, clean=True)
    distdf['Index'] = distdf.index
    noNANdf['Index'] = noNANdf.index
    tmp_df = pd.merge(noNANdf, distdf,how='inner')
    tmp_df.to_csv(analysedAllelesPath, sep='\t')
    
    print("Done with caculate pairwise alignment distances.")
    #pd.util.testing.assert_frame_equal(alleleDf, pd.read_csv(analysedAllelesPath, sep='\t', index_col=0))
    print("Starting to parse dN/dS ratios from yn.out file.")
    dNdS_suffix = 'dNdSdf_tmp'
    #now assign the dNdS ratios
    Parallel(n_jobs=threads)(delayed(assign_dNdS_to_all_alleles)(list(folder_index_list),all_folders,tmp_path, dNdS_suffix)\
                       for folder_index_list in grouper(noNANdf.index, 100, np.nan))
    dNdS_header = ['yn00_dN/dS','LWL85_dN/dS','LWL85m_dN/dS','LPB93_dN/dS' ]
    dNdSdf = combineTmpToDf(dNdS_header, dNdS_suffix, tmp_path, clean=True)
    dNdSdf['Index'] = dNdSdf.index
    tmp_df = pd.merge(tmp_df, dNdSdf,how='inner')
    tmp_df.to_csv(analysedAllelesPath, sep='\t')
    print('Finish to parse dN/dS ratios.')
    print('Combining Dataframe of potential allele pairs with Dataframe of blast none hits.')
    #now pull together the dataframes of hits and no-hits blast again.
    pre_col_order = tmp_df.columns
    alleleDf = pd.concat([tmp_df, alleleDf.loc[alleleDf.index.isnull(), :]], axis=0, ignore_index=True)
    alleleDf = alleleDf.loc[:,pre_col_order]
    
    print("Assigning the unphased genes.")
    unphased_s = assign_unphased(alleleDf.Query)
    assert(any(unphased_s.index == alleleDf.Query))
    alleleDf['unphased'] = unphased_s.values
    
    print("Assining the proteinortho single genes.")
    PO_single_s = assign_PO_single(alleleDf.Query)
    assert(any(PO_single_s.index == alleleDf.Query))
    alleleDf['PO_single'] = PO_single_s.values
    
    print("Assining the proteinortho inter haplome paralogoues by comparing\
    proteinortho (po) and its synteny mode output (poff)")
    alleleDf['aTarget+aQuery'] = alleleDf.aTarget + '+' + alleleDf.aQuery
    interhaplotypeparaloges_s = assign_INP(alleleDf['aTarget+aQuery'])
    assert(any(interhaplotypeparaloges_s.index == alleleDf['aTarget+aQuery'] ))
    alleleDf['PO_interhaplotype_paralogs'] = interhaplotypeparaloges_s.values
    alleleDf.drop('aTarget+aQuery', axis=1, inplace=True)
    alleleDf.reset_index(drop=True, inplace=True)
    alleleDf.to_csv(analysedAllelesPath, sep='\t')
    return alleleDf

In [152]:
if __name__ == "__main__":
    alleleDf_out = main()

Checking whether all PAML files already exist in /home/benjamin/genome_assembly/Warrior/allele_analysis/v04/post_analysis/paml...
PAML appears to have been run to completion previously. Therefore, it will not be run this time.
203
Done with caculate pairwise alignment distances.
Starting to parse dN/dS ratios from yn.out file.
203
Finish to parse dN/dS ratios.
Combining Dataframe of potential allele pairs with Dataframe of blast none hits.
Assigning the unphased genes.
Assining the proteinortho single genes.
Assining the proteinortho inter haplome paralogoues by comparing    proteinortho (po) and its synteny mode output (poff)
Check poff and proteinortho output as conceptually all poff pairs should be in included in the proteinortho output.
These are the pairs:
['evm.model.hcontig_056_002.2+evm.model.pcontig_058.1', 'evm.model.hcontig_050_006.10+evm.model.pcontig_050.56', 'evm.model.hcontig_036_004.26+evm.model.pcontig_036.110']


In [153]:
assert(len(alleleDf) == len(alleleDf_out))

In [156]:
alleleDf_out[(alleleDf_out.unphased == True) & (alleleDf_out.PO_single == True)]

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,...,protein_levenshtein,cds_hamming,cds_levenshtein,yn00_dN/dS,LWL85_dN/dS,LWL85m_dN/dS,LPB93_dN/dS,unphased,PO_single,PO_interhaplotype_paralogs
39,evm.model.pcontig_000.1068,evm.model.hcontig_000_054.9,79.67,246.0,24.0,1.0,1.0,246.0,1.0,220.0,...,0.8336,0.8245,0.8245,0.4342,0.4009,0.8530,0.7245,True,True,False
61,evm.model.pcontig_000.121,evm.model.hcontig_014_010.9,27.85,237.0,142.0,10.0,284.0,507.0,2274.0,2494.0,...,0.9094,0.8784,0.8513,0.1855,NaN,NaN,NaN,True,True,False
62,evm.model.pcontig_000.124,evm.model.hcontig_009_030.33,40.38,52.0,30.0,1.0,94.0,145.0,39.0,89.0,...,0.8234,0.7542,0.6929,0.2376,NaN,NaN,NaN,True,True,False
131,evm.model.pcontig_000.226,evm.model.hcontig_040_007.41,27.90,534.0,318.0,15.0,38.0,552.0,4.0,489.0,...,0.8321,0.7624,0.7114,0.1878,0.3450,0.3083,0.3876,True,True,False
133,evm.model.pcontig_000.228,evm.model.hcontig_000_048.30,57.37,190.0,62.0,5.0,205.0,394.0,8.0,178.0,...,0.7387,0.6999,0.6809,0.5743,0.4815,0.5134,0.5319,True,True,False
213,evm.model.pcontig_000.339,evm.model.hcontig_038_008.14,85.94,128.0,15.0,2.0,1.0,126.0,1.0,127.0,...,0.1603,0.1323,0.1323,0.4089,0.2594,0.4050,0.3666,True,True,False
215,evm.model.pcontig_000.340,evm.model.hcontig_000_031.19,28.98,245.0,127.0,10.0,154.0,361.0,196.0,430.0,...,0.8427,0.7720,0.7191,0.2170,NaN,NaN,NaN,True,True,False
219,evm.model.pcontig_000.350,evm.model.hcontig_008_038.9,76.39,72.0,17.0,0.0,1.0,72.0,159.0,230.0,...,0.9417,0.9350,0.9346,0.4800,0.2859,0.4339,0.4172,True,True,False
220,evm.model.pcontig_000.351,evm.model.hcontig_001_024.7,29.82,218.0,123.0,5.0,172.0,362.0,2778.0,2992.0,...,0.9534,0.9366,0.9232,0.1858,NaN,NaN,NaN,True,True,False
223,evm.model.pcontig_000.356,evm.model.hcontig_058_009.51,50.24,422.0,159.0,8.0,96.0,505.0,76.0,458.0,...,0.7040,0.6764,0.6451,0.1077,0.1513,0.2024,0.1919,True,True,False


In [160]:
alleleDf_out[(alleleDf_out.unphased == False) & (alleleDf_out.PO_single == True)]['Query'].unique().shape[0]

6404

In [166]:
alleleDf_out.allele_source.unique()

array(['BLAST', 'PO', 'h_rBLAST', nan], dtype=object)

In [174]:
phFullAlleleDf.columns

Index(['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery',
       'StopQuery', 'StartTarget', 'StopTarget', 'e-value', 'BitScore',
       'QLgth', 'QCov', 'TLgth', 'TCov', 'q_contig', 't_contig',
       'q_contig == t_contig', 'p_protein', 'h_contig_overlap',
       't_contig == h_contig_overlap', 'allele_source', 'matchType', 'aQuery',
       'aTarget'],
      dtype='object')

In [172]:
alleleDf_out[alleleDf_out.allele_source == 'PO'].matchType

11188    PO
11189    PO
11190    PO
11191    PO
11192    PO
11193    PO
11194    PO
11195    PO
11196    PO
11197    PO
11198    PO
11199    PO
11200    PO
11201    PO
11202    PO
11203    PO
11204    PO
11205    PO
11206    PO
11207    PO
11208    PO
11209    PO
11210    PO
11211    PO
11212    PO
11213    PO
11214    PO
11215    PO
11216    PO
11217    PO
         ..
19300    PO
19301    PO
19302    PO
19303    PO
19304    PO
19305    PO
19306    PO
19307    PO
19308    PO
19309    PO
19310    PO
19311    PO
19312    PO
19313    PO
19314    PO
19315    PO
19316    PO
19317    PO
19318    PO
19319    PO
19320    PO
19321    PO
19322    PO
19323    PO
19324    PO
19325    PO
19326    PO
19327    PO
19328    PO
19329    PO
Name: matchType, Length: 8142, dtype: object

In [ ]:
########## FIGURE PLOTTING ##########

def make_autopct(values):
    def my_autopct(pct):
        total = sum(values)
        val = int(round(pct*total/100.0))
        return '{p:.2f}%\n({v:d})'.format(p=pct,v=val)
    return my_autopct

def autolabel(rects, labels, ax, fontsize):
    """
    Attach a text label above each bar displaying its height
    """
    for i, rect in enumerate(rects):
        height = rect.get_height()
        ax.text(rect.get_x() + rect.get_width()/2., height, str(labels[i]), ha='center', va='bottom', fontsize=fontsize)

def getNumNoAlleles(pProteinFastaFile, alleleDf):
    with open(pProteinFastaFile) as pProteinFasta:
        pProteinList = []
        for line in pProteinFasta:
            if line.startswith('>'):
                pProteinList.append(line[1:].strip())

    assert(len(pProteinList) == len(set(pProteinList)))


    pairedPProteinList = list(alleleDf['Query'])
    pairedPProteinList += list(alleleDf['Target'])
    pairedPProteinList = set(pairedPProteinList)
    
    for pairedPProtein in pairedPProteinList:
        if pairedPProtein in pProteinList:
            pProteinList.remove(pairedPProtein)
    
    return len(pProteinList)

def plotAlleleTypesPie(ax, alleleDf, colors, includeNoAlleles=True):
    '''Plots a pie chart of allele types, with the option of also including 
    primary proteins with no alleles. Strictly, this is not an accurate representation
    of the distribution of primary proteins as the reciprocal BLAST-identified (h on p) alleles
    may result in double-counting of primary proteins.
    '''
    # OrderedDict to preserve order, so that plots are coloured with same key as the distance 
    # bar graphs. This is a bit of a hack-fix; must enter these by hand again in the same order 
    # as 'matchType' occurs in the alleleAveragesByMatchType DataFrame.
    alleleTypeCountDict = collections.OrderedDict()
    
    for matchType in alleleDf['matchType'].unique():
        alleleTypeCountDict[matchType] = len(alleleDf[alleleDf['matchType'] == matchType])
    
    if includeNoAlleles==True:
        numNoAlleles = getNumNoAlleles(P_PROTEINS_FASTA, alleleDf)
        alleleTypeCountDict['no_allele'] = numNoAlleles

    patches, texts, autotexts = ax.pie(list(alleleTypeCountDict.values()), labels=alleleTypeCountDict.keys(), autopct=make_autopct(list(alleleTypeCountDict.values())), colors=colors)
    ax.axis('equal')
    ax.set_title('Allele Types', loc='center', fontsize=TITLE_SIZE, position=(0.5, 1.1))

def plotLevenshteinBar(alleleAverages, ax, colors):
    '''Plots a bar graph of normalised Levenshtein distances on ax from DataFrame alleleAverages.'''
    
    ind = np.arange(len(alleleAverages.protein_levenshtein))
    rects = ax.bar(ind, alleleAverages.protein_levenshtein, 0.35, color=colors, align='center') 
    
    sns.despine(top=True, right=True)
    
    barLabels = []
    for levDist in alleleAverages.protein_levenshtein:
        barLabels.append(str(int((1-levDist)*100)) + '%')
    autolabel(rects, barLabels, ax, INLINE_LABEL_SIZE)

    ax.set_xticks(ind)
    ax.set_xticklabels(alleleAverages.index, rotation=45)

    # ax.set_xlabel('Allele Types', fontsize=AXIS_LABEL_SIZE)
    ax.set_ylabel('Normalised Levenshtein Distance', fontsize=AXIS_LABEL_SIZE)

    ax.tick_params(axis='both', which='major', labelsize=AXIS_TICK_SIZE, pad=3)

    for tick in ax.get_xaxis().get_major_ticks():
        tick.set_pad(2*tick.get_pad())
        tick.label1 = tick._get_text1()
        
def plotAlleles(alleleDf, qCovFilters, tCovFilters, pctIdFilters, levSimFilters, leavePO):
    '''Makes a 3x2 plot with normalised Levenshtein distance plots in column 1 and
    a pie chart representing the distribution of allele types in column 2.
    Each row shows different levels of filtering.
    
    leavePO is a boolean that determines whether only BLAST hits will be filtered (leavePO=True)
    or both BLAST and PO alleles should be filtered (leavePO=False)'''
    cmap = plt.cm.Greens
    colors = cmap(np.linspace(0.0, 0.6, len(alleleDf['matchType'].unique())))
    
    assert(len(qCovFilters) == len(pctIdFilters) == len(levSimFilters))
    
    fig, ax = plt.subplots(len(qCovFilters), 2, figsize=(30, 12*len(qCovFilters)))
    
    for i in range(len(qCovFilters)):
        
        filteredAlleleDf = filterAlleleDf(alleleDf, qCovFilters[i], tCovFilters[i], pctIdFilters[i], levSimFilters[i], True)
        print(filteredAlleleDf[filteredAlleleDf.allele_source == 'PO'].shape[0])
        # levenshtein distance plot
        alleleAveragesByMatchType = filteredAlleleDf.groupby(['matchType']).mean()
        plotLevenshteinBar(alleleAveragesByMatchType, ax[i, 0], colors)
        ax[i, 0].set_xticklabels(alleleAveragesByMatchType.index, rotation=45, ha='right')
        
        # pie plot
        plotAlleleTypesPie(ax[i, 1], filteredAlleleDf, colors)
        
        # include filtering criteria in title
        qCovFilter = qCovFilters[i]
        tCovFilter = tCovFilters[i]
        pctIdFilter = pctIdFilters[i]
        levSimFilter = levSimFilters[i]
        if qCovFilter < BASE_QCOV_CUTOFF:
            qCovFilter = BASE_QCOV_CUTOFF
            print('Base QCov cut-off is 70%; if you desire to filter below this value, decrease BASE_QCOV_CUTOFF.')
        if tCovFilter < BASE_TCOV_CUTOFF:
            tCovFilter = BASE_TCOV_CUTOFF
            print('Base TCov cut-off is 70%; if you desire to filter below this value, decrease BASE_TCOV_CUTOFF.')
        if pctIdFilter < BASE_PCTID_CUTOFF:
            pctIdFilter = BASE_PCTID_CUTOFF
            print('Base %ID cut-off is 70%; if you desire to filter below this value, decrease BASE_PCTID_CUTOFF.')
        if not levSimFilter:
            levSimFilter = 0
    
        ax[i, 0].set_title('QCov > %s%%, TCov > %s%%, ID > %s%%, L. sim. > %s%%, PO Filtered: %s' % (qCovFilter, tCovFilter, pctIdFilter, levSimFilter, not leavePO), position=(0.5, 0.85))
        
    fig.tight_layout()
    fig.savefig(os.path.join(FIGURE_PATH, 'fig'), bbox_inches='tight')

In [ ]:
# Used in the pie chart for all text except title
# the ax.pie plotting interface is weird - cannot set other font sizes properly?
mpl.rcParams['font.size'] = 24

TITLE_SIZE = 32
AXIS_LABEL_SIZE = 28
AXIS_TICK_SIZE = 24
INLINE_LABEL_SIZE = 24

# These lists must all be the same length
qCovFilters = [False, 80, 90, 95]
pctIdFilters = [False, 80, 90, 95]
tCovFilters = [False, 80, 90, 95]
levSimFilters = [False, False, False, False]

plotAlleles(alleleDf, qCovFilters, tCovFilters, pctIdFilters, levSimFilters, True)